In [ ]:
!pip install spacy -U 
!python -m spacy download pl_core_news_lg

In [2]:
import spacy
import pandas as pd
import re
import matplotlib.pyplot as plt

In [379]:
nlp = spacy.load("pl_core_news_lg")
nlp_blank = spacy.blank("pl")

Część 1 - przetwarzanie i czyszczenie danych

In [380]:
df_twitter = pd.read_csv("/content/ani_jednej_wiecej.csv")

df_created_at = df_twitter["created_at"]
df_user_created_at = df_twitter["user_created_at"]
df_tweet_url = df_twitter["tweet_url"]
df_urls = df_twitter["urls"]
df_img_url = df_twitter["media"]
df_text = df_twitter["text"]

In [ ]:
#1 zamiana nazw dni tygodnia

days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
days_pl = ["Poniedziałek", "Wtorek", "Środa", "Czwartek", "Piątek", "Sobota", "Niedziela"]

df_twitter["created_at"] = df_created_at.replace(days, days_pl, regex = True)
df_created_at

# df_twitter

In [ ]:
#2 zamiana nazw miesięcy

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
months_num = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

df_user_created_at = df_user_created_at.replace(months, months_num, regex = True)
df_user_created_at

# df_twitter

In [ ]:
#3 lista linków tweetów

lista_linkow = [link for link in df_tweet_url]
lista_linkow


In [ ]:
#4 lista z linkami udostępnionymi w postach

shared_url = [str(link) for link in df_urls] 
shared_url = [elem for elem in shared_url if elem != "nan"]

print(shared_url)


In [ ]:
#5 lista z linkami do obrazków

img_url = [str(link) for link in df_img_url]
img_url = [elem for elem in img_url if elem != "nan"] #usuwanie "nan" z listy

print(img_url)

In [ ]:
#6 usuwanie stopwords

stopwords = nlp.Defaults.stop_words

df_twitter['text_without_stopwords'] = df_twitter['text'].apply(lambda words: ' '.join([word for word in words.split() if word.lower() not in (stopwords)]))

print(df_text, "\n")
print(df_twitter['text_without_stopwords'])


Część 2 - eksploracyjna analiza danych

In [390]:
df_fav = df_twitter["favorite_count"]
df_retweet = df_twitter["retweet_count"]
df_sensitive = df_twitter["possibly_sensitive"]
df_user_date = df_twitter["user_created_at"]
df_followers = df_twitter["user_followers_count"]
df_name = df_twitter["user_name"]
df_verified = df_twitter["user_verified"]

In [ ]:
#1 top5 największa ilość polubień

#sortuję listę od największej wartości do najmniejszj
top5_fav = df_fav.sort_values(ascending=False).head()

top = 0
for index, number in top5_fav.items():
  top +=1
  print(f"""
  Top {top}
  Liczba polubień {number}: 
  {df_text[index]}""")

In [ ]:
#2 top5 największa ilość retweetow

#sotuję listę i usuwam duplikaty (top 5 było powieleniem tego samego tweeta)
top5_retweet = df_retweet.sort_values(ascending=False).drop_duplicates().head()

top = 0
for index, number in top5_retweet.items():
  top +=1
  print(f"""
  Top {top}
  Liczba retweetów: {number}
  {df_text[index]}""")
  

In [ ]:
#3 Wyświetlanie tweetów nieuznanych jako wrażliwych

not_sensitive = df_sensitive.where(df_sensitive == False).dropna()

for index, value in not_sensitive.items():
  tweet = df_text[index]
  print(index, "\n", tweet, "\n")
  

In [396]:
#4 użytkownik, który najwcześniej założył konto

#stworzyłam nową kolumnę, w której przechowywany jest rok założenia konta
#oprócz roku, wyszukiwana była też fraza '0000' występująca przed rokiem, dlatego istotny jest tylko jeden element z wyszukanej listy
df_twitter["birth_year"] = [re.findall("\d{4}", elem)[1] for elem in df_user_date]

#okazało się, że tylko jeden użytkownik założył konto w 2006 roku i nie było potrzeby sprawdzania różnicy w miesiącach i dniach
year_created = df_twitter["birth_year"].sort_values().head(1)

for index, year in year_created.items():
    print(f"""
    {df_name[index]}
    rok założenia konta: {year}
    {df_text[index]}"""
    )



    Hex 🇵🇱 #PolskiŁad #MuremZaPolskimMundurem
    rok założenia konta: 2006
    #AniJednejWięcej. Jak politycy manipulują przepisami i treścią wyroku TK - https://t.co/MuUX62CedH https://t.co/yn3o7MuNsm


In [ ]:
#5 tweety użytkownika z największą ilością followersów

user_max = df_followers.max()

for index, followers in df_followers.items():
  if followers == user_max:
    print(f"""
    {df_name[index]} ({user_max} followersów)
    {df_text[index]}
    """)


In [ ]:
#6 wwyświetlanie zweryfikowanych użytkowników

verified_users_index = df_verified.where(df_verified == True).dropna()
# print(verified_users_index)

verified_users = [df_name[index] for index, verification in verified_users_index.items()]
# print(verified_users)

#tworzę listę bez duplikatów
users_no_duplicates = []

for name in verified_users:
  if name not in users_no_duplicates:
    users_no_duplicates.append(name)
    print(name)

In [ ]:
#7 najpopularniejszy dzień tygodnia publikowania

week = {
    "poniedziałek": df_created_at.str.count(r'Poniedziałek').sum(),
    "wtorek": df_created_at.str.count(r'(Wtorek)').sum(),
    "środa": df_created_at.str.count(r'(Środa)').sum(),
    "czwartek": df_created_at.str.count(r"(Czwartek)").sum(),
    "piątek": df_created_at.str.count(r'(Piątek)').sum(),
    "sobota": df_created_at.str.count(r'(Sobota)').sum(),
    "niedziela": df_created_at.str.count(r'(Niedziela)').sum()
    }

# print(week)

all_values = week.values()
max_value = max(all_values)

for day, value in week.items():
  if value == max_value:
    print(f"""
    Dzień tygodnia, w którym zostało opublikowanych najwięcej tweetów: {day}
    liczba tweetów: {value}""")


Część 3 - przetwarzanie języka naturalnego

In [420]:
doc_1 = [nlp(text) for text in df_text]

In [ ]:
#2 rozpoznawanie osób

people = []

#lista excluded_people powstała po analizie wykrytych słów
excluded_people = ['jprdl https://t.co/RZ6epYW0a1', 'https://t.co/BmEKSFfV3g', 'https://t.co/RZilLymkzp', 'https://t.co/PuBPs7XP6T', '✍️', 'https://t.co/lWOqq1gcEY', 'https://t.co/LZecWajv4b' ]

for tweet in doc_1:
  osoba = ""
  for ent in tweet.ents:
    if ent.label_ == "persName" and ent.text not in excluded_people:
      osoba = osoba + ent.text + " "
  people.append(osoba)

assert len(people) == df_twitter.shape[0]
df_twitter["persons"] = people

# people_values = df_twitter["persons"].value_counts()
# print(people_values[0:50])

print(df_twitter['persons'])


In [ ]:
#2 rozpoznawanie miejsc

places = []

#zmienna excluded_people powstała po analizie wykrytych słów
excluded_places = "ŻE" #zwroty, które nie są miejscami a były identyfikowane

for tweet in doc_1:
  miejsce = ""
  for ent in tweet.ents:
    if ent.label_ == "placeName" and ent.text != excluded_places:
      miejsce = miejsce + ent.text + " " 
  places.append(miejsce)

assert len(places) == df_twitter.shape[0]
df_twitter["places"] = places

# places_values = df_twitter["places"].value_counts()
# print(places_values[0:50])

print(df_twitter['places'][0:50])


In [ ]:
#3 rozpoznawanie organizacji

org = []

for tweet in doc_1:
  organization = ""
  for ent in tweet.ents:
    if ent.label_ == "orgName":
      organization = organization + ent.text + " " 
  org.append(organization)

assert len(org) == df_twitter.shape[0]
df_twitter["organizations"] = org

# org_values = df_twitter["organizations"].value_counts()
# print(org_values[0:50])

print(df_twitter["organizations"])



In [ ]:
#4 rozpoznawanie hashtagów

ruler = nlp_blank.add_pipe("entity_ruler")

pattern = "\S+"
patterns = [{"label": "hashtag", "pattern": [{"ORTH": "#"}, {"TEXT": {"REGEX": pattern}}]}]

ruler.add_patterns(patterns)

In [425]:
doc_2 = [nlp_blank(text) for text in df_twitter["text"]]

In [ ]:
hashtags = []

for tweet in doc_2:
  hashtag = ""
  for ent in tweet.ents:
    if ent.label_ == "hashtag":
      hashtag = hashtag + ent.text + " " 
  hashtags.append(hashtag)

assert len(hashtags) == df_twitter.shape[0]
df_twitter["hashtags"] = hashtags

# hash_values = df_twitter["hashtags"].value_counts()
# print(hash_values)

print(df_twitter['hashtags'])

Część 4 - rozwiązywanie problemów/umiejętność interpretowania dokumentacji

In [ ]:
#wykres liczba tweetów per dni tygodnia

plt.figure(figsize=(10, 8))
plt.title("Liczba tweetów opublikowanych według dni tygodnia", fontsize = 20)
plt.xlabel('Dni tygodnia', fontsize = 14)
plt.ylabel('Liczba tweetów', fontsize = 14)
plot = plt.bar(*zip(*week.items()), color = 'g', width = 0.5)